In [1]:
# module import
import tensorflow as tf
import numpy as np

* 입력 이미지의 채널은 컨볼루션 필터, 풀링 레이어, 업샘플링에 동일하게 적용됨.
* 풀링 레이어의 수와 필터의 수는 같아야 함.(필터로 나온 만큼 풀링 적용해야 하기 때문)


In [2]:
# 이미지, 필터, 풀링, 업샘플링 정의
n_batch, n_height, n_width, n_channel = (1, 5, 5, 1) # 입력 이미지: 1장, 5x5, 흑백.
f_height, f_width, f_channel, f_count = (3, 3, n_channel, 1) # 컨볼루션 필터: 1장, 3x3, 흑백.
p_height, p_width, p_channel, p_count = (3, 3, n_channel, f_count) # 풀링: 1장, 3x3 사이즈.
u_height, u_width, u_channel, u_count = (3, 3, n_channel, f_count) # 업샘플링: 1장, 3x3 사이즈.

In [9]:
# 이미지, 필터, 업샘플 레이어 정의
X = np.arange(0.1, 2.6, 0.1).reshape(n_batch, n_height, n_width, n_channel) # 0.1부터 2.6까지의 픽셀 5x5 사이즈 이미지로 가정.
F = np.random.rand(9).reshape(f_height, f_width, f_channel, f_count)
U = np.array(np.repeat(1, 9)).reshape(u_height, u_width, u_channel, u_count) # untrainable, 상수 업샘플링 레이어.

In [11]:
# 이미지 확인
print("========== Input Image ==========")
print(X) # 원래 Keras layer에 들어가는 형태
print(X.reshape(n_height, n_width)) # 2차원 이미지 형태로 표현

========== Input Image ==========
[[[[0.1]
   [0.2]
   [0.3]
   [0.4]
   [0.5]]

  [[0.6]
   [0.7]
   [0.8]
   [0.9]
   [1. ]]

  [[1.1]
   [1.2]
   [1.3]
   [1.4]
   [1.5]]

  [[1.6]
   [1.7]
   [1.8]
   [1.9]
   [2. ]]

  [[2.1]
   [2.2]
   [2.3]
   [2.4]
   [2.5]]]]
[[0.1 0.2 0.3 0.4 0.5]
 [0.6 0.7 0.8 0.9 1. ]
 [1.1 1.2 1.3 1.4 1.5]
 [1.6 1.7 1.8 1.9 2. ]
 [2.1 2.2 2.3 2.4 2.5]]


In [12]:
# 컨볼루션 필터 확인
print("========== Convolution Filter ==========")
print(F.reshape(f_height, f_width)) # 2차원 형태로 표현

========== Convolution Filter ==========
[[0.48234635 0.92134703 0.13204006]
 [0.38135477 0.98996817 0.84678709]
 [0.09391369 0.89831192 0.51753341]]


In [13]:
# 업샘플 필터 확인
print("========== Upsampling Filter ==========")
print(U.reshape(u_height, u_width)) # 2차원 형태로 표현

========== Upsampling Filter ==========
[[1 1 1]
 [1 1 1]
 [1 1 1]]


In [15]:
# CNN 예시
image = tf.constant(X, dtype=tf.float32)
filter = tf.constant(F, dtype=tf.float32)
conv_layer = tf.nn.conv2d(image, filter, strides=[1,1,1,1], padding='VALID') # 패딩 설정 안함.
pooling_layer = tf.nn.max_pool(conv_layer, ksize=[1, p_height, p_width, 1], strides=[1,1,1,1], padding='SAME') # 패딩 적용함: 이미지 크기 유지
upsampling_layer = tf.nn.conv2d_transpose(pooling_layer, U, output_shape=[1, n_height, n_width, 1], strides=[1,1,1,1], padding='VALID') # 패딩 적용 안 함.

In [18]:
# Convolution Layer 확인
print("========== Convolution Layer ==========")
print(conv_layer.numpy().reshape(n_height-f_height+1, n_width-f_width+1)) # padding 적용 안 함. convolution filter 공식에 의해

========== Convolution Layer ==========
[[ 3.725409   4.2517695  4.7781296]
 [ 6.35721    6.8835707  7.40993  ]
 [ 8.989012   9.515372  10.041732 ]]


In [20]:
# Pooling Layer 확인
print("========== Pooling Layer ==========")
print(pooling_layer.numpy().reshape(f_height, f_width)) # padding 적용됨.

========== Pooling Layer ==========
[[ 6.8835707  7.40993    7.40993  ]
 [ 9.515372  10.041732  10.041732 ]
 [ 9.515372  10.041732  10.041732 ]]


In [21]:
# Upsampling Layer 확인
print("========== Upsampling Layer ==========")
print(upsampling_layer.numpy().reshape(n_height, n_width)) # padding 적용: 원래 이미지 크기로 돌아감.

========== Upsampling Layer ==========
[[ 6.8835707 14.293501  21.70343   14.81986    7.40993  ]
 [16.398943  33.850605  51.30227   34.903324  17.451662 ]
 [25.914314  53.40771   80.90111   54.98679   27.493393 ]
 [19.030745  39.11421   59.197674  40.166927  20.083464 ]
 [ 9.515372  19.557104  29.598835  20.083464  10.041732 ]]
